In [1]:
%%js
Jupyter.notebook.get_cell(0).element.hide()
Jupyter.notebook.get_cell(1).element.hide()
Jupyter.notebook.get_cell(25).element.hide()
Jupyter.notebook.get_cell(26).element.hide()

<IPython.core.display.Javascript object>

In [2]:

import pandas as pd


# Annual Prediction 

In [3]:
train2 = pd.read_csv("Bihis 2020.csv")

In [4]:
train1 = pd.read_csv("Bihis 2021.csv")

In [5]:
test = pd.read_csv("Bihis 2022.csv")

In [6]:
train2_len = len(train2)
train1_len = len(train1)
test_len = len(test)

In [7]:
df = pd.concat([train2, train1, test], axis=0)
pd.set_option('display.max_rows', df.shape[0]+1)
df.reset_index(drop=True, inplace=True)
df

,Name,Size,Cost,Category,Sales,Year
0,Kamias,Medium,100,Tree,3400.0,2020
1,Kalamasi,Medium,120,Tree,4080.0,2020
2,Papaya,Small,50,Tree,1700.0,2020
3,Tundan,Small,80,Tree,3200.0,2020
4,Lakatan,Small,150,Tree,10500.0,2020
5,Rambutan,Medium,250,Tree,5000.0,2020
6,Suha,Small,130,Tree,1950.0,2020
7,Atis,Medium,130,Tree,3770.0,2020
8,Guyabano,Medium,100,Tree,2200.0,2020
9,Lemon,Large,100,Tree,1700.0,2020


In [8]:
%%capture
print(len(train2), len(train1), len(test), len(df))

In [9]:
%%capture
df.apply(lambda x: len(x.unique()))

In [10]:
%%capture
df.isnull().sum()

In [11]:
%%capture
cat_col = []
for x in df.dtypes.index:
    if df.dtypes[x] == 'object':
        cat_col.append(x)
cat_col

In [12]:
%%capture
cat_col.remove('Name')
cat_col

In [13]:
%%capture
for col in cat_col:
    print(col)
    print(df[col].value_counts())
    print();

In [14]:
%%capture
size_mode = df.pivot_table(values = 'Size', columns = 'Category', aggfunc=(lambda x: x.mode()[0]))
size_mode

In [15]:
miss_bool = df['Size'].isnull()
df.loc[miss_bool, 'Size'] = df.loc[miss_bool, 'Category'].apply(lambda x: size_mode[x])

In [16]:
%%capture
df['Size'].isnull().sum()

In [17]:
df['Total_Year'] = 2022 - df['Year']

In [18]:
%%capture
df['Total_Year']

In [19]:
%%capture
df = pd.get_dummies(df, columns=[ 'Size', 'Category'])
df.head();

In [20]:
%%capture
df.tail();

In [21]:
%%capture
train2 = df.iloc[:train2_len,:]
train1 = df.iloc[train2_len:,:]
test = df.iloc[train2_len:,:]

In [22]:
%%capture
test = test.drop(columns=['Sales'])
test.head()

In [23]:
%%capture
test.tail();

In [24]:
X = train2.drop(columns=['Year', 'Name', 'Sales'])
y = train2['Sales']

In [25]:
from sklearn.metrics import mean_squared_error
def train2(model, X, y):
    model.fit(X, y)
    model.score(X, y)
    

In [26]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
train2(model, X, y)
coef = pd.Series(model.feature_importances_, X.columns).sort_values(ascending=False)

In [27]:
%%capture
x_test = test.drop(columns=['Year', 'Name'])
x_test.head();

# Prediction

In [28]:
class RandomForestRegresor:
    def __init__(self, n_trees=10, max_depth=3, min_samples_split=2, n_feature=None):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.n_features = n_feature
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            tree = DecisionTree(max_depth=self.max_depth,
                                   min_samples_split=self.min_samples_split,
                                   n_features=self.n_features)
            X_sample, y_sample = self._bootstrap_samples(X, y)
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)
        
    def _bootstrap_samples(self, X, y):
        n_samples = X.shape[0]
        idxs = np.random.choice(n_samples, n_samples, replace=True)
        return X[idxs], y[idxs]
    
    def _most_common_label(self, y):
        counter = Counter(y)
        most_common = counter.most_common(1)[0][0]
        return most_common

    def predict(self, X):
        predictions = np.array([tree.predict(X) for tree in self.trees])
        return predictions


In [29]:
pred = model.predict(X)
print("Model Report Score:", model.score(X, y))
print("Accuracy:", mean_squared_error(y,pred))



Model Report Score: 0.7813850908734932
Accuracy: 918871.032218639


In [30]:
pred = model.predict(x_test)
pred

array([2692.89047619, 3779.54      , 1956.33333333, 2806.13333333,
       7047.46107143, 3557.3       , 2271.66178571, 3547.515     ,
       2692.89047619, 1487.65714286, 2271.66178571, 2692.89047619,
       2947.73571429, 2271.66178571, 3127.54      , 1487.65714286,
       2271.66178571, 3557.3       , 2889.        , 6291.86666667,
       4423.8       , 4713.6       , 2177.46666667, 1884.        ,
       2987.3       , 2434.73333333, 1732.3       , 2454.66666667,
       7047.96107143, 3127.54      , 2692.89047619, 3779.54      ,
       1956.33333333, 2806.13333333, 7047.46107143, 3557.3       ,
       2271.66178571, 3547.515     , 2692.89047619, 1487.65714286,
       2271.66178571, 2692.89047619, 2947.73571429, 2271.66178571,
       3127.54      , 1487.65714286, 2271.66178571, 3557.3       ,
       2889.        , 6291.86666667, 4423.8       , 4713.6       ,
       2177.46666667, 1884.        , 2987.3       , 2434.73333333,
       1732.3       , 2454.66666667, 7047.96107143, 3127.54   

# Annual Prediction result

In [31]:
mission = pd.DataFrame()
mission['Name'] = test["Name"]
mission['sales'] = pred
mission['Year'] = 2022
pd.set_option('display.max_rows', df.shape[0]+1)
mission


,Name,sales,Year
28,Kamias,2692.890476,2022
29,Kalamasi,3779.540000,2022
30,Papaya,1956.333333,2022
31,Tundan,2806.133333,2022
32,Lakatan,7047.461071,2022
33,Rambutan,3557.300000,2022
34,Suha,2271.661786,2022
35,Atis,3547.515000,2022
36,Guyabano,2692.890476,2022
37,Lemon,1487.657143,2022


# Quarterly Prediction

In [32]:
mission = pd.DataFrame()
mission['Name'] = test["Name"]
mission['sales'] = pred / 12
mission['Quarterly'] = 3
pd.set_option('display.max_rows', df.shape[0]+1)
mission

,Name,sales,Quarterly
28,Kamias,224.407540,3
29,Kalamasi,314.961667,3
30,Papaya,163.027778,3
31,Tundan,233.844444,3
32,Lakatan,587.288423,3
33,Rambutan,296.441667,3
34,Suha,189.305149,3
35,Atis,295.626250,3
36,Guyabano,224.407540,3
37,Lemon,123.971429,3


# Monthly Prediction

In [33]:
mission = pd.DataFrame()
mission['Name'] = test["Name"]
mission['sales'] = pred / 36
mission['monthly'] = 1
pd.set_option('display.max_rows', df.shape[0]+1)
mission

,Name,sales,monthly
28,Kamias,74.802513,1
29,Kalamasi,104.987222,1
30,Papaya,54.342593,1
31,Tundan,77.948148,1
32,Lakatan,195.762808,1
33,Rambutan,98.813889,1
34,Suha,63.101716,1
35,Atis,98.542083,1
36,Guyabano,74.802513,1
37,Lemon,41.323810,1
